# Refinitiv Data Library for Python
## Delivery - OMMItemStream - Market By Price data via callback

This notebook demonstrates how to use the OMM Item Stream interface to request streaming Full Depth Orderbook data

## Import the library and load credentials

Credentials used by this and the other tutorials notebooks are stored in the **Configuration/credentials.ipynb** file.     

You should have edited the **Configuration/credentials.ipynb** to set your credentials as part of the **Quick Start** step.

In [1]:
from refinitiv.data.delivery import omm_stream
import datetime
import json
%run ../../Configuration/credentials.ipynb

## Open the session of your choice

Use our helper function in the Credentials notebook, **open_session(session_type)**, to create and open a session to connect to the 
- Refinitiv Data Platform directly (session_type="rdp") or via 
- Eikon 4 or Refinitiv Workspace (session_type="desktop") or via a 
- local realtime infrastructure (session_type="deployed").

You can also set a default in the **credentials** notebook

In [2]:
open_session()

<refinitiv.data.session._platform_session.Definition object at 0x2b06d534190 {session_name='default-session'}>

# Example

## Define a function to display events

In [3]:
# Callback function to display data or status events
def display_event(eventType, event):
    currentTime = datetime.datetime.now().time()
    print("----------------------------------------------------------")
    print(">>> {} event received at {}".format(eventType, currentTime))
    # Print only 1000 characters to limit the output
    print(json.dumps(event, indent=2)[0:1000])
    return

## Create an Item Stream and register the event callbacks

In [4]:
stream = omm_stream.Definition(name="VOD.L", domain='MarketByPrice').get_stream()

# Refresh - the first full imaage we get back from the server
stream.on_refresh(lambda item_stream, event : display_event("Refresh", event))
# Update - as and when field values change, we receive updates from the server
stream.on_update(lambda item_stream, event : display_event("Update", event))
# Status - if data goes stale or item closes, we get a status message
stream.on_status(lambda item_stream, event : display_event("Status", event))
# Other errors
stream.on_error(lambda item_stream, event : display_event("Error", event))

## Open the Item Stream
The **open()** call to open the Item Stream is a synchronous one. This means the first event (either via on_refresh(), on_status() or on_error()) can occur before the open() method returns.   
However, if we use the **open_async()** asynchronous method instead, the first event callback will be after  open_async() returns.

In [5]:
# Library will request OrderBook from server
stream.open()
# We should intially receive the full orderbook, 
# after which we will receive updates for specific order (Add, Update, Delete)
# Note from the above display_event() function that I am dumping just the first 1000 characters to minimise output
# You should remove this limit to see the full response.

[2021-09-03 11:00:14,434] - [INFO] - [default-session] - [27236] | WebSocket 0 - OMM Protocol - PRICING
Login to websocket wss://apac-3-t3.streaming-pricing-api.refinitiv.com:443/WebSocket successful
----------------------------------------------------------
>>> Refresh event received at 11:00:14.948765
{
  "ID": 1,
  "Type": "Refresh",
  "Domain": "MarketByPrice",
  "Key": {
    "Service": "ELEKTRON_DD",
    "Name": "VOD.L"
  },
  "State": {
    "Stream": "Open",
    "Data": "Ok",
    "Text": "**All is well"
  },
  "Complete": false,
  "Qos": {
    "Timeliness": "Realtime",
    "Rate": "JitConflated"
  },
  "PermData": "AwEBJJw=",
  "SeqNumber": 37792,
  "Map": {
    "KeyType": "Buffer",
    "Summary": {
      "Fields": {
        "PROD_PERM": 249,
        "DSPLY_NAME": "VODAFONE GROUP",
        "CURRENCY": "GBp",
        "ACTIV_DATE": "2021-09-03",
        "LOT_SIZE_A": 1,
        "RECORDTYPE": 113,
        "SEQNUM": 4344310,
        "RDN_EXCHD2": "LSE",
        "PROV_SYMB": "133215",

<StreamState.Open: 3>

----------------------------------------------------------
>>> Refresh event received at 11:00:15.130885
{
  "ID": 1,
  "Type": "Refresh",
  "Domain": "MarketByPrice",
  "Key": {
    "Service": "ELEKTRON_DD",
    "Name": "VOD.L"
  },
  "State": {
    "Stream": "Open",
    "Data": "Ok",
    "Text": "**All is well"
  },
  "Complete": false,
  "Qos": {
    "Timeliness": "Realtime",
    "Rate": "JitConflated"
  },
  "ClearCache": false,
  "PermData": "AwEBJJw=",
  "SeqNumber": 37792,
  "Map": {
    "KeyType": "Buffer",
    "Entries": [
      {
        "Action": "Add",
        "Key": "MTI3Ljk4QQ==",
        "Fields": {
          "ORDER_PRC": 127.98,
          "ORDER_SIDE": "ASK",
          "NO_ORD": 1,
          "ACC_SIZE": 275000,
          "LV_TIM_MS": 14404009
        }
      },
      {
        "Action": "Add",
        "Key": "MTEwLjI2Qg==",
        "Fields": {
          "ORDER_PRC": 110.26,
          "ORDER_SIDE": "BID",
          "NO_ORD": 1,
          "ACC_SIZE": 2200,
          "LV_T

You may notice that there are multiple Refresh events - this is often the case for the more actively traded instruments with large orders books.
Once all Refresh events have been received, you can then expect to receive Update events with just the Order changes i.e. Add, Update or Delete orders.  
**NOTE:** I am truncating the output to the 1st 1000 character of each response payload - for ease of viewing.  


## Close Stream

In [6]:
stream.close()

<StreamState.Closed: 1>

### Close Session

In [7]:
close_session()